In [4]:
from io import StringIO
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf
import time
import numpy as np
import warnings
import re

warnings.filterwarnings('ignore', category=DeprecationWarning)

def get_stock_holdings_and_pledge(stock_symbol):
    """
    Fetches FII, DII, Public, Promoters, and extracts pledge % with improved parsing.
    """
    url = f"https://www.screener.in/company/{stock_symbol.upper()}/"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        pledge_latest = np.nan
        holdings_section = soup.find('section', id='shareholding')
        
        # Primary: Broad search for "Pledged" in page text
        page_text = soup.get_text(separator=' ', strip=True)
        pledge_match = re.search(r'(?:Pledged|pledge)\s*percentage?\s*[:\-]?\s*(\d+(?:\.\d+)?)%?', page_text, re.IGNORECASE)
        if pledge_match:
            pledge_latest = float(pledge_match.group(1))
            print(f"  Pledge found in page text: {pledge_latest}%")
        
        # Secondary: Specific ratios section
        if np.isnan(pledge_latest):
            ratios_section = soup.find('section', {'id': 'ratios'} ) or soup.find('div', class_=re.compile(r'ratios|overview'))
            if ratios_section:
                section_text = ratios_section.get_text(separator=' ', strip=True)
                pledge_match = re.search(r'(?:Pledged|pledge)\s*[:\-]?\s*(\d+(?:\.\d+)?)%?', section_text, re.IGNORECASE)
                if pledge_match:
                    pledge_latest = float(pledge_match.group(1))
                    print(f"  Pledge found in ratios section: {pledge_latest}%")
        
        # Tertiary: Holdings table Promoters row (fallback)
        if np.isnan(pledge_latest) and holdings_section:
            html_string = str(holdings_section)
            data = StringIO(html_string)
            tables = pd.read_html(data)
            if tables and len(tables) > 0:
                df = tables[0]
                mask = df['Unnamed: 0'].str.contains('FII|DII|Public|Promoters', case=False, na=False)
                filtered_df = df[mask].copy()
                if filtered_df.empty:
                    return None, 0.0  # Default low
                filtered_df = filtered_df.rename(columns={'Unnamed: 0': 'Category'})
                filtered_df['Category'] = filtered_df['Category'].str.replace(r'\s*\+\s*', '', regex=True).str.strip()
                
                quarter_cols = [col for col in filtered_df.columns if col != 'Category']
                if not quarter_cols:
                    return filtered_df, 0.0
                for col in quarter_cols:
                    filtered_df[col] = pd.to_numeric(filtered_df[col].astype(str).str.replace('%', ''), errors='coerce')
                
                promoter_row = filtered_df[filtered_df['Category'].str.contains('Promoters', na=False)]
                if not promoter_row.empty and len(quarter_cols) > 0:
                    latest_col = quarter_cols[0]
                    cell_text = str(promoter_row[latest_col].iloc[0])
                    cell_match = re.search(r'pledged?\s*[:\-]?\s*(\d+(?:\.\d+)?)%?', cell_text, re.IGNORECASE)
                    if cell_match:
                        pledge_latest = float(cell_match.group(1))
                        print(f"  Pledge found in table cell: {pledge_latest}%")
        
        # Final default: If still NaN, assume 0 for common cases (log warning)
        if np.isnan(pledge_latest):
            pledge_latest = 0.0
            print(f"  Warning: Pledge not found for {stock_symbol}; defaulting to 0.0%")
        
        # Return holdings DF (even if partial)
        if holdings_section:
            html_string = str(holdings_section)
            data = StringIO(html_string)
            tables = pd.read_html(data)
            if tables and len(tables) > 0:
                df = tables[0]
                mask = df['Unnamed: 0'].str.contains('FII|DII|Public|Promoters', case=False, na=False)
                filtered_df = df[mask].copy()
                if not filtered_df.empty:
                    filtered_df = filtered_df.rename(columns={'Unnamed: 0': 'Category'})
                    filtered_df['Category'] = filtered_df['Category'].str.replace(r'\s*\+\s*', '', regex=True).str.strip()
                    quarter_cols = [col for col in filtered_df.columns if col != 'Category']
                    for col in quarter_cols:
                        filtered_df[col] = pd.to_numeric(filtered_df[col].astype(str).str.replace('%', ''), errors='coerce')
                    return filtered_df, pledge_latest
        return None, pledge_latest
        
    except Exception as e:
        print(f"Scraping error for {stock_symbol}: {e}")
        return None, 0.0  # Graceful default

def check_fii_dii_increase(df):
    """Checks FII/DII increase QoQ or YoY, with safe handling."""
    if df is None or df.empty or len(df) < 3:
        return False, False
    quarter_cols = [col for col in df.columns if col != 'Category']
    if len(quarter_cols) < 4:
        return False, False
    fii_row = df[df['Category'] == 'FIIs']
    dii_row = df[df['Category'] == 'DIIs']
    latest, prev_q, prev_y = quarter_cols[0], quarter_cols[1], quarter_cols[3]
    
    fii_inc = False
    if not fii_row.empty:
        try:
            fii_latest = fii_row[latest].iloc[0]
            if pd.notna(fii_latest):
                fii_prev_q = fii_row[prev_q].iloc[0] if prev_q in fii_row.columns else np.nan
                fii_prev_y = fii_row[prev_y].iloc[0] if prev_y in fii_row.columns else np.nan
                fii_inc = (pd.notna(fii_prev_q) and fii_latest > fii_prev_q) or (pd.notna(fii_prev_y) and fii_latest > fii_prev_y)
        except (IndexError, KeyError):
            pass
    
    dii_inc = False
    if not dii_row.empty:
        try:
            dii_latest = dii_row[latest].iloc[0]
            if pd.notna(dii_latest):
                dii_prev_q = dii_row[prev_q].iloc[0] if prev_q in dii_row.columns else np.nan
                dii_prev_y = dii_row[prev_y].iloc[0] if prev_y in dii_row.columns else np.nan
                dii_inc = (pd.notna(dii_prev_q) and dii_latest > dii_prev_q) or (pd.notna(dii_prev_y) and dii_latest > dii_prev_y)
        except (IndexError, KeyError):
            pass
    
    return fii_inc, dii_inc

# yfinance with Robust Checks
def get_yf_financials(ticker):
    stock = yf.Ticker(ticker)
    income_stmt = stock.quarterly_financials.T if hasattr(stock, 'quarterly_financials') and not stock.quarterly_financials.empty else pd.DataFrame()
    balance_sheet = stock.quarterly_balance_sheet.T if hasattr(stock, 'quarterly_balance_sheet') and not stock.quarterly_balance_sheet.empty else pd.DataFrame()
    info = stock.info or {}
    return income_stmt, balance_sheet, info

def apply_financial_filters(ticker):
    """Applies filters with fixed debt-to-equity calculation."""
    income_stmt, balance_sheet, info = get_yf_financials(ticker)
    if income_stmt.empty or len(income_stmt) < 4 or balance_sheet.empty:
        print(f"  Insufficient quarterly data for {ticker}")
        return None

    result = {}

    # [Revenue, Net Profit, ROE, ROCE, EPS sections unchanged from previous fixed version]

    # Updated Debt to Equity <1 with multiple fallbacks and safe access
    debt_to_equity_lt_1 = False
    debt_val = np.nan
    equity_val = np.nan

    # Equity first (prioritize keys)
    equity_candidates = ['Total Stockholder Equity', 'Total Equity', 'Common Stock Equity']
    for eq_key in equity_candidates:
        eq_series = balance_sheet.get(eq_key, pd.Series(dtype=float))
        if not eq_series.empty and len(eq_series) > 0:
            equity_val = eq_series.iloc[0]
            break

    # Debt: Sum components or use direct
    if not np.isnan(equity_val) and equity_val != 0:
        # Primary: Sum Long Term + Short Long Term Debt
        long_debt_series = balance_sheet.get('Long Term Debt', pd.Series(dtype=float))
        short_debt_series = balance_sheet.get('Short Long Term Debt', pd.Series(dtype=float))
        if not long_debt_series.empty and len(long_debt_series) > 0 and not short_debt_series.empty and len(short_debt_series) > 0:
            debt_val = long_debt_series.iloc[0] + short_debt_series.iloc[0]
        else:
            # Fallback: Total Debt
            total_debt_series = balance_sheet.get('Total Debt', pd.Series(dtype=float))
            if not total_debt_series.empty and len(total_debt_series) > 0:
                debt_val = total_debt_series.iloc[0]

        # If still no debt from sheet, use info fallback (annual, but better than nothing)
        if np.isnan(debt_val):
            debt_eq_info = info.get('debtToEquity', np.nan)
            if not np.isnan(debt_eq_info):
                # debtToEquity is already the ratio, so use it directly
                debt_to_equity_lt_1 = debt_eq_info < 1
                result['debt_to_equity_lt_1'] = debt_to_equity_lt_1
                result['debt_to_equity_value'] = debt_eq_info  # For debugging
                return result  # Early return if using info

        # Calculate ratio if debt found
        if not np.isnan(debt_val):
            debt_eq = debt_val / equity_val
            debt_to_equity_lt_1 = debt_eq < 1
            result['debt_to_equity_value'] = debt_eq  # Debug value

    result['debt_to_equity_lt_1'] = debt_to_equity_lt_1
    if np.isnan(debt_to_equity_lt_1):
        print(f"  Warning: Could not compute D/E for {ticker}; defaulting to False")

    return result


# Main Function (Unchanged logic, but uses fixed apply_financial_filters)
def fundamental_stock_filter(tickers):
    results = []
    for ticker in tickers:
        yf_ticker = f"{ticker}.NS"
        print(f"Processing {ticker}...")

        yf_filters = apply_financial_filters(yf_ticker)
        if yf_filters is None:
            print(f"  Skipping {ticker}: Insufficient yfinance data")
            continue

        holdings_df, pledge_latest = get_stock_holdings_and_pledge(ticker)
        fii_inc, dii_inc = check_fii_dii_increase(holdings_df)
        promoter_pledge_ok = (pledge_latest < 10) if not np.isnan(pledge_latest) else True  # True if default 0

        core_passes = all([
            yf_filters.get(k, False) for k in [
                'revenue_gt_prev_quarter', 'revenue_gt_prev_year', 'net_profit_positive',
                'net_profit_gt_prev_quarter', 'net_profit_gt_prev_year', 'roe_gt_10',
                'roce_gt_10', 'eps_gt_0', 'eps_gt_prev_quarter', 'eps_gt_prev_year',
                'debt_to_equity_lt_1'
            ]
        ])
        all_pass = core_passes and promoter_pledge_ok and fii_inc and dii_inc

        row = {'Ticker': ticker, 'Passes_All': all_pass, **yf_filters,
               'Promoter_Pledge_OK': promoter_pledge_ok, 'FII_Increasing': fii_inc,
               'DII_Increasing': dii_inc, 'Pledge_%': pledge_latest}
        results.append(row)
        time.sleep(2)  # Rate limit

    return pd.DataFrame(results)

# Example Usage
tickers = ['RELIANCE', 'TCS', 'INFY']  # Your array
filtered_df = fundamental_stock_filter(tickers)
print("\nFilter Results:")
print(filtered_df.round(4))  # Rounded for readability

passing_stocks = filtered_df[filtered_df['Passes_All'] == True]['Ticker'].tolist()
print(f"\nStocks passing all filters: {passing_stocks}")

# Optional export
# filtered_df.to_csv('fundamental_filter_results_fixed.csv', index=False)


Processing RELIANCE...
Processing TCS...
Processing INFY...

Filter Results:
     Ticker  Passes_All  debt_to_equity_value  debt_to_equity_lt_1  \
0  RELIANCE       False                0.4383                 True   
1       TCS       False                0.0982                 True   
2      INFY       False                0.0626                 True   

   Promoter_Pledge_OK  FII_Increasing  DII_Increasing  Pledge_%  
0                True            True           False       0.0  
1                True            True           False       0.0  
2                True            True           False       0.0  

Stocks passing all filters: []
